In [1]:
import io
import os
import cv2
import asyncio
import concurrent
import numpy as np
from pyastrovis.webrtc import WebRTCStream
from pyastrovis.widget import WebRTCWidget
from pyastrovis.fits import FITSImageCubeStream
from ipywidgets import IntSlider, HBox, VBox
import bqplot.pyplot as plt
from IPython.display import display
import uvloop

uvloop.install()

In [2]:
server = await WebRTCWidget.create_server()

In [3]:
fits = await FITSImageCubeStream.open('/Users/dpallot/Projects/jacal/summit_demo/oskar/ingest/big.fits', num_processes=4)

In [4]:
panel = server.create_panel(256, 256)
x = [i for i in range(fits.get_num_channels())]
y = [await fits.get_spectral_line(5, 3)]
axes_options = {'x': {'label': 'Channel'}, 'y': {'label': 'Jy/beam'}}
fig = plt.figure()
fig.layout.height = '300px'
fig.layout.width = '700px'
lines = plt.plot(x=x, y=y, colors=['red'], axes_options=axes_options)

In [5]:
loop = asyncio.get_running_loop()
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

async def channel():
    while True:
        value = await wait_for_change(slider, 'value')
        data = await fits.get_channel_data(value)
        norm_img = 255*((data-data.min())/data.ptp())
        norm_img = norm_img.astype(np.uint8)
        img = cv2.applyColorMap(norm_img, cv2.COLORMAP_HSV)
        await panel.add_data(img, 'rgb24')

async def spectra():
    while True:
        x, y = await wait_for_change(panel, 'position')
        spec = await fits.get_spectral_line(x, y)
        lines.y = spec
        
asyncio.ensure_future(channel())
asyncio.ensure_future(spectra())

slider = IntSlider(value=0, min=0, step=1, max=fits.get_num_channels()-1)


display(HBox([panel, fig]), slider)

IntSlider(value=0, max=34619)

In [ ]:
panel.get_position()

In [ ]:
await panel.close()
await server.close()
await fits.close()